In [ ]:
import cv2
import glob
import numpy as np
from keras.preprocessing.image import random_rotation

eno_X_train = []

for f in glob.glob("image(えのき茸と松茸)/train/エノキ/*.jpg"):

  img=cv2.imread(f)/255.0
  #img=cv2.imread(f)
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  eno_X_train.append(img_resized)               # numpy型データをリストに追加

  for i in range(9):
      arr2 = img_resized
      arr2 = random_rotation(arr2, rg=360)
      eno_X_train.append(arr2) 


eno_X_train= np.array(eno_X_train)

In [ ]:
matu_X_train = []

for f in glob.glob("image(えのき茸と松茸)/train/松茸/*.jpg"):

  img=cv2.imread(f)/255.0
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  matu_X_train.append(img_resized)               # numpy型データをリストに追加

  for i in range(9):
      arr2 = img_resized
      arr2 = random_rotation(arr2, rg=360)
      matu_X_train.append(arr2) 


matu_X_train= np.array(matu_X_train)

In [ ]:
X_train = np.concatenate([eno_X_train, matu_X_train], axis=0)
y_train = np.concatenate([np.ones((len(eno_X_train),1)), np.zeros((len(matu_X_train), 1))], axis=0)#えのき茸だと1,松茸だと0と指定

In [ ]:
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential
from keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras

# Densenetのapi(https://keras.io/ja/applications/#densenet)
model = Sequential()
model.add(DenseNet121(weights='imagenet', include_top=False, input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Dense(128, activation='ReLU'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(128, activation='ReLU'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(128, activation='ReLU'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(64, activation='ReLU'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])#学習モデル精度の設定(このように学習してくださいという指定)

es = EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=30,
    callbacks=[es],
    validation_split=0.2
)

In [ ]:
eno_X_test = []

for f in glob.glob("image(えのき茸と松茸)/test/えのき茸/*.jpg"):

  img=cv2.imread(f)/255.0
  #img=cv2.imread(f)
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  eno_X_test.append(img_resized)               # numpy型データをリストに追加

eno_X_test= np.array(eno_X_test)

matu_X_test = []

for f in glob.glob("image(えのき茸と松茸)/test/松茸/*.jpg"):

  img=cv2.imread(f)/255.0
  #img=cv2.imread(f)
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  matu_X_test.append(img_resized)               # numpy型データをリストに追加


matu_X_test= np.array(matu_X_test)


X_test = np.concatenate([eno_X_test, matu_X_test], axis=0)
y_test = np.concatenate([np.ones((len(eno_X_test),1)), np.zeros((len(matu_X_test), 1))], axis=0)#エノキだと1,松茸だと0と指定

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=y_pred[:, 0, 0, 1])

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

In [ ]:
import numpy as np
correct = []
incorrect=[]


for num in range(16): 
  #print(f'{num+1}''枚目')
  print(int(num)+1,'枚目')
  
  if np.argmax(y_pred[int(num)])==y_test[int(num)]:
      print('予測一致')
      
      correct.append(int(1))

  else :
      print('予測不一致')
      incorrect.append(int(1))

print('     ')

print('予測が正解した合計枚数')
print(len(correct))

print('予測が不正解だった合計枚数')
print(len(incorrect))

print('     ')
print('正解率')
rate=(int(len(correct))/(int(len(correct))+int(len(incorrect))))*100

print(rate,'%')